In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
from model import CropYieldModel, CropDataset


## 데이터 전처리 및 로딩

In [4]:
# 데이터 전처리 및 로딩
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = CropDataset('json_directory/', 'images_directory/', transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


Creating dataset from json_directory/ and images_directory/
Found 50 json files
Loaded 50 valid samples out of 50 JSON files


## 모델, 손실 함수, 옵티마이저 초기화 

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CropYieldModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 학습루프

In [6]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for i, (images, features, labels) in enumerate(dataloader):
        images = images.to(device)
        features = features.to(device)
        labels = labels.to(device)

        outputs = model(images, features)
        loss = criterion(outputs, labels.unsqueeze(1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 169413.9688
Epoch [2/100], Loss: 125347.8203
Epoch [3/100], Loss: 81046.8281
Epoch [4/100], Loss: 29065.8242
Epoch [5/100], Loss: 48622.3125
Epoch [6/100], Loss: 36061.4961
Epoch [7/100], Loss: 16742.9258
Epoch [8/100], Loss: 43301.5312
Epoch [9/100], Loss: 45336.9883
Epoch [10/100], Loss: 28670.4512
Epoch [11/100], Loss: 18483.6270
Epoch [12/100], Loss: 13587.4111
Epoch [13/100], Loss: 3603.4604
Epoch [14/100], Loss: 3281.8430
Epoch [15/100], Loss: 25226.0508
Epoch [16/100], Loss: 13711.6445
Epoch [17/100], Loss: 10452.7627
Epoch [18/100], Loss: 5933.5132
Epoch [19/100], Loss: 10463.6396
Epoch [20/100], Loss: 3053.0608
Epoch [21/100], Loss: 8879.2979
Epoch [22/100], Loss: 9926.3721
Epoch [23/100], Loss: 10743.1289
Epoch [24/100], Loss: 2891.8062
Epoch [25/100], Loss: 3155.7275
Epoch [26/100], Loss: 4928.4565
Epoch [27/100], Loss: 4344.0381
Epoch [28/100], Loss: 3334.7830
Epoch [29/100], Loss: 6898.8774
Epoch [30/100], Loss: 3582.7842
Epoch [31/100], Loss: 4463.212

In [7]:
torch.save(model.state_dict(), 'crop_yield_model.pth')

In [8]:
model.eval()
with torch.no_grad():
    new_image_id = "0120200824MS02N000070"  # 예시 ID
    
    # 새 JSON 파일 읽기
    with open(f"json_directory/{new_image_id}.json", 'r', encoding='utf-8-sig') as f:
        new_data = json.load(f)
    
    # 새 이미지 파일 찾기 (jpg만)
    new_image_jpg = f"images_directory/{new_image_id}.jpg"
    
    if os.path.exists(new_image_jpg):
        new_image_path = new_image_jpg
    else:
        raise FileNotFoundError(f" 이미지 {new_image_id} 파일이 없습니다.")
    
    new_image = transform(Image.open(new_image_path).convert('RGB')).unsqueeze(0).to(device)
    new_features = torch.tensor([[
        new_data['GSD'],
        new_data['LONG'],
        new_data['LAT'],
        new_data['GROWTH_1'],
        new_data['GROWTH_2']
    ]], dtype=torch.float32).to(device)
    
    prediction = model(new_image, new_features)
    print(f'예측된 수확량 : {prediction.item():.2f}')

Predicted Yield: 279.58
